In [1]:
import sys
import os
sys.path.append("../")
DATA_DIR = "../dataset/training/"

In [2]:
from scripts.preprocessing import *
from scripts.augmentation import *

ModuleNotFoundError: No module named 'preprocessing'

CREATE AUGMENTED DATASET

In [ ]:
ORIGIN_IMAGES = DATA_DIR + "images/"
ORIGIN_MASKS = DATA_DIR + "groundtruth/"
AUG_IMAGES = DATA_DIR + "augmented_images/"
AUG_MASKS = DATA_DIR + "augmented_groundtruth/"
FILES = sorted(os.listdir(ORIGIN_IMAGES))

In [ ]:
# Load original dataset
images = [load_image(ORIGIN_IMAGES + FILES[i]) for i in range(len(FILES))]
masks = [load_image(ORIGIN_MASKS + FILES[i]) for i in range(len(FILES))]
print(len(images), len(masks))

100 100


In [3]:
if not os.path.isdir(AUG_IMAGES):
    os.mkdir(AUG_IMAGES)
if not os.path.isdir(AUG_MASKS):
    os.mkdir(AUG_MASKS)

NameError: name 'AUG_IMAGES' is not defined

In [4]:
# Generates the 700 new images. The 100 first images are the original images
data_augmented(images, masks, AUG_IMAGES, AUG_MASKS)

NameError: name 'data_augmented' is not defined